In [2]:
# 화면 가로 확장 코드 (기본 width 50%)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:70% !important; }</style>"))

### 데이터 판다스로 입력, 컬럼 정리

In [1]:
import pandas as pd
ch1 = pd.read_csv('./channel_1.dat', names=['Unix', 'Watt'],header=None, delimiter=" ")
print('Module Import Done.')

FileNotFoundError: [Errno 2] No such file or directory: './channel_1.dat'

In [ ]:
ch1.info()

In [ ]:
from datetime import datetime
ch1_date = pd.DataFrame(ch1,columns = ['Unix', 'Date', 'Watt'])

ch1_date['Date'] = pd.to_datetime(ch1['Unix'], unit='s')
ch1_date.info()
ch1_date.head(10)

### 변수 및 구분 단계

In [ ]:
"""
jump 를 하는 경우가 있기 때문에 이 경우를 구분 짓는 변수를 j로 선택

watt_list 에는 해당 시단위동안 모든 watt값들이 저장됨. 그중에서 max값을 추출하면 max_watt로 저장됨.

n은 한시간 동안의 데이터 개수를 세기 위한 변수

d4는 정상적으로 있어야 하는 일자인데 하루가 통째로 비는 경우를 처리하는 변수 D는 d4+1로 첫 생성

H1은 데이터 원본에 존재하는 시각만 불러오지만, H0는 결손 여부와 무관하게 하루 시각 전체를 표시. (0~23)

# 1. 시작 지점 지정

# 2. 전체 반복 While문 진입 (13.1.1 ~ 16.12.31)

    # 3. 하루 단위 반복 while문 
    
        # 3'. 결손 데이터를 처리하고 하루를 넘기기 위한 하루 반복문의 이중 While문
        
            # 3.1. 하루 데이터의 시작이 0시부터 정상적으로 존재하는 경우
            
                # 3.1'. h시 ~ h+1시 사이에서 반복되는 1시간 단위 코드의 큰 틀
                
                    # 3.1'.1. 시 정보가 일치할때
                                6, 7초 간격으로 존재하는 데이터들을 변환하고 리스트에 모두 저장
                    # 3.1'.2. 시 단위가 바로 다음시 즉, h시 일때 그 다음이 h+1이 되는 경우.
                                1시간 동안 저장된 전력중 맥스값을 추출하고 csv 파일 저장
                    # 3.1'.3. 23시까지 정리를 마치고 다음 일자로 넘어가는 경우.
                                1시간 동안 맥스값을 저장하고 break로 탈출, 일자 변경
                    # 3.1'.4. 바로 다음시가 아닌 결손 데이터로 인해 시가 건너뛰는 경우
                                한 시간 동안의 데이터 수는 600개를 넘지 않기 때문에 조건으로 설정.
                                우선 해당 시각 존재하는 만큼의 데이터는 정리해서 맥스값 추출, 저장
                        
                        # 3.1'.4.1. 같은 일자 안에서 건너뛰는 경우
                                    점프된 시각을 변수로 기록, 점프된 시각 전까지 해당 시는 Null로 저장
                        # 3.1'.4.2. 결손 데이터가 지속되다가 일자가 달라지는 경우
                                    해당 일의 잔여시를 처리해서 Null로 저장
                            
                            # 3.1'.4.2.1. 일자가 달라지는데 하루 차이가 아닌 경우 = 특정일이 통째로 결손.
                                            결손 된후의 일자까지 Null 처리, 저장
                                            
                                # 3.1'.4.2.1.1. 월 중간에서 데이터가 빈 경우
                                                결손 이후 다음 일자 전까지 Null 처리
                                # 3.1'.4.2.1.2. 월말에 데이터가 비어서 다음달로 넘어갈 때까지 빈 경우.
                                                결손 일자+1부터 당월 말일까지 (Dict에 일자 정의) Null처리
                
                # 3.2. 하루의 데이터가 0시부터 존재하는 것이 아닌 결손되어 있는경우
                        데이터가 존재하는 시 전까지 Null 저장
                    
"""

### csv 파일 저장을 위한 폴더 생성

In [5]:
import os
try:
    os.mkdir('./Electirc_Data_CSV')
except FileExistsError:
    print('해당 폴더가 이미 존재합니다.\n폴더를 확인해주세요.')
    

해당 폴더가 이미 존재합니다.
폴더를 확인해주세요.


### 데이터 변환 및 CSV 파일 저장 코드

### 중간 정보 확인창 출력 없이 시작, 종료 메세지만 출력

In [ ]:
import csv
import time

month = {1:31, 2:28, 3:31, 4:30, 5:31, 6:30, 7:31, 8:31, 9:30, 10:31, 11:30, 12:31}

# 1. 
i = 431369#431369가 맞음
print('Running!', '\n'*4)


# 2. 
while(i < 20284837):
    try:
        list1 = ch1_date.loc[i].values.tolist()
        date = list1[1]
        y1 = date.year
        M1 = date.month
        d1 = date.day
        H1 = date.hour 
        n = 0 


        # 3. 
        while(H1 < 24):
            H0 = 0 

            # 3'. 
            while(H0 < 24):
                watt_list=[]
                h = date.hour

                # 3.1. 
                if(H1 == H0):

                    # 3.1'. 
                    while(h < h+1):
                        n=n+1
                        list2 = ch1_date.loc[i].values.tolist()
                        date = list2[1]
                        watt = list2[2]
                        y = date.year
                        M = date.month
                        d = date.day
                        h = date.hour
                        m = date.minute
                        s = date.second

                        j=h-H1

                        # 3.1'.1. 
                        if(j==0):
                            watt_list.append(watt)
                            i = i+1
                            """
                            여기서 1시간 내의 과정이 이뤄지니 그안에서 5분이든 10분이든 처리를 해줘야 함.
                            """

                        # 3.1'.2.
                        elif (j==1):
                            max_watt = max(watt_list)

                            hourly_date2 = '{0}년 {1}월 {2}일 - {3}시'.format(y, M, d, H1)
                            hourly_date5 = '{0} {1} {2}'.format(y, M, d) 
                            hourly_list = []
                            hourly_list.append(hourly_date2)
                            """
                            밑에 있는 hourly_list는 1시간 단위로 max_watt의 값을 가져오는 것이니 여기서도 주기에 맞게 처리를 해줘야함
                            
                            밑에 있는 max_watt 등장 부분 모두 마찬가지.
                            """
                            hourly_list.append(max_watt)

                            with open("./Electirc_Data_CSV/"+str(y)+'_'+str(M)+'_'+"House1_Ch1_Electric_Data.csv", 'a', newline='') as file:
                                writer = csv.writer(file)
                                writer.writerow(hourly_list)       

                            H1 = H1+1 
                            H0 = H0+1
                            break

                        # 3.1'.3. 
                        elif(d1+1==d and h ==0 and H1 ==23):
                            d=d-1
                            h=h+24

                            max_watt = max(watt_list)

                            hourly_date3 = '{0}년 {1}월 {2}일 - {3}시'.format(y, M, d, H1)
                            hourly_list = []
                            hourly_list.append(hourly_date3)
                            hourly_list.append(max_watt)

                            with open("./Electirc_Data_CSV/"+str(y)+'_'+str(M)+'_'+"House1_Ch1_Electric_Data.csv", 'a', newline='') as file:
                                writer = csv.writer(file)
                                writer.writerow(hourly_list)       

                            H1 = H1+1 
                            H0 = H0+1                   
                            break

                        # 3.1'.4.
                        elif (n>600):
                            list(hourly_date5)
                            listed = hourly_date5.split()
                            y4 = listed[0]
                            M4 = listed[1]
                            d4 = listed[2]

                            max_watt = max(watt_list)

                            hourly_date = '{0}년 {1}월 {2}일 - {3}시'.format(y4, M4, d4, H1)
                            hourly_list = []
                            hourly_list.append(hourly_date)
                            hourly_list.append(max_watt)

                            with open("./Electirc_Data_CSV/"+str(y4)+'_'+str(M4)+'_'+"House1_Ch1_Electric_Data.csv", 'a', newline='') as file:
                                writer = csv.writer(file)
                                writer.writerow(hourly_list)       

                            # 3.1'.4.1. 
                            if(d1==d):
                                jc=j

                                while(jc>1):
                                    h=h+1
                                    hourly_date = '{0}년 {1}월 {2}일 - {3}시'.format(y4, M4, d4, h-j)
                                    hourly_list = []
                                    hourly_list.append(hourly_date)
                                    hourly_list.append('Null')

                                    with open("./Electirc_Data_CSV/"+str(y4)+'_'+str(M4)+'_'+"House1_Ch1_Electric_Data.csv", 'a', newline='') as file:
                                        writer = csv.writer(file)
                                        writer.writerow(hourly_list)  

                                    jc=jc-1
                                    H1 = H1+1
                                    H0 = H0+1

                                H1 = H1+1
                                H0 = H0+1

                            # 3.1'.4.2.
                            if(d1!=d):

                                while(H1<23):
                                    hourly_date = '{0}년 {1}월 {2}일 - {3}시'.format(y4, M4, d4, H1+1)
                                    hourly_list = []
                                    hourly_list.append(hourly_date)
                                    hourly_list.append('Null')

                                    with open("./Electirc_Data_CSV/"+str(y4)+'_'+str(M4)+'_'+"House1_Ch1_Electric_Data.csv", 'a', newline='') as file:
                                        writer = csv.writer(file)
                                        writer.writerow(hourly_list)  

                                    H1 = H1+1
                                    H0 = H0+1

                                H1 = H1+1
                                H0 = H0+1

                                # 3.1'.4.2.1. 
                                if(int(d4)+1 != d):
                                    D = int(d4)+1 

                                    # 3.1'.4.2.1.1. 
                                    if(int(M) == int(M4)):
                                        while(D<d):
                                            for x in range (0,24):
                                                hourly_date = '{0}년 {1}월 {2}일 - {3}시'.format(y4, M4, D, x)
                                                hourly_list = []
                                                hourly_list.append(hourly_date)
                                                hourly_list.append('Null')

                                                with open("./Electirc_Data_CSV/"+str(y4)+'_'+str(M4)+'_'+"House1_Ch1_Electric_Data.csv", 'a', newline='') as file:
                                                    writer = csv.writer(file)
                                                    writer.writerow(hourly_list)  

                                            D=D+1

                                    # 3.1'.4.2.1.2.
                                    if(int(M) != int(M4)):
                                        while(D<=month[int(M4)]):
                                            for x in range (0,24):
                                                hourly_date = '{0}년 {1}월 {2}일 - {3}시'.format(y4, M4, D, x)
                                                hourly_list = []
                                                hourly_list.append(hourly_date)
                                                hourly_list.append('Null')

                                                with open("./Electirc_Data_CSV/"+str(y4)+'_'+str(M4)+'_'+"House1_Ch1_Electric_Data.csv", 'a', newline='') as file:
                                                    writer = csv.writer(file)
                                                    writer.writerow(hourly_list)  

                                            D=D+1

                                        D2 = 1
                                        while(D2 < d):
                                            for x in range (0,24):
                                                hourly_date = '{0}년 {1}월 {2}일 - {3}시'.format(y4, M, D2, x)
                                                hourly_list = []
                                                hourly_list.append(hourly_date)
                                                hourly_list.append('Null')

                                                with open("./Electirc_Data_CSV/"+str(y4)+'_'+str(M)+'_'+"House1_Ch1_Electric_Data.csv", 'a', newline='') as file:
                                                    writer = csv.writer(file)
                                                    writer.writerow(hourly_list)  

                                            D2 = D2+1

                            break


                # 3.2. 
                if(H1 != H0):
                    list3 = ch1_date.loc[i].values.tolist()
                    date = list3[1]
                    watt = list3[2]
                    y = date.year
                    M = date.month
                    d = date.day
                    h = date.hour
                    m = date.minute
                    s = date.second

                    j=h-H1           

                    hourly_date3121 = '{0}년 {1}월 {2}일 - {3}시'.format(y, M, d, H0)
                    hourly_list3121 = []
                    hourly_list3121.append(hourly_date3121)
                    hourly_list3121.append('Null')

                    with open("./Electirc_Data_CSV/"+str(y)+'_'+str(M)+'_'+"House1_Ch1_Electric_Data.csv", 'a', newline='') as file:
                        writer = csv.writer(file)
                        writer.writerow(hourly_list3121)       

                    h = h+1
                    H0 = H0+1

                    if (H0 ==23 and h!=24 and H1!=23):
                                break

                    if(h==24 and H0==23 and H1==23):
                        hourly_date3121 = '{0}년 {1}월 {2}일 - 23시'.format(y, M, d)
                        hourly_list3121 = []
                        hourly_list3121.append(hourly_date3121)
                        hourly_list3121.append('Null')

                        with open("./Ch"+str(Total_num)+"_Electirc_Data_CSV/"+str(y)+'_'+str(M)+'_'+"House1_Ch"+str(Total_num)+"_Electric_Data.csv", 'a', newline='') as file:
                            writer = csv.writer(file)
                            writer.writerow(hourly_list3121)

            break
    except:
        pass
    
print('Job Done!!')
time.sleep(5)